# <center>  PROYECTO FINAL <center> 

<center> Diplomado en Ciencia de Datos <center>
<center> Modulo 1  <center>
<center> 27 de agosto de 2024 <center>


##  EQUIPO 1 
- Abner Colli Sanchez
- María Dolores Matus Basto 
- Samuel Muñoz Villarino 

##  Objetivo del Estudio
 En el contexto de la ciencia de datos aplicada al análisis de datos, la FIFA nos ha otorgado la tabla de jugadores (players) y la tabla de atributos de los jugadores (player attributes) que proporcionan información valiosa sobre los futbolistas de las ligas europeas y sus características.
 
 Desarrollamos un Modelo Predictivo utilizando modelos de regresión logisitica  para predecir qué jugadores tienen más probabilidades de ser seleccionados para el Salón de la Fama basándose en sus atributos durante su carrera.
 
 Proporcionaremos un Informe para el Comité de Selección detallando las recomendaciones sobre los mejores candidatos para el Salón de la Fama, respaldado por datos y análisis exhaustivos. Este enfoque integral permite tomar decisiones informadas sobre qué jugadores que merecen ser reconocidos en el Salón de la Fama, utilizando una combinación de análisis estadístico de sus atributos como jugador y su comparación histórica.




In [1]:
### 
import pandas as pd
import numpy as np
import sqlite3

## Base de datos

#### Los datos utilizados provienen de la base de datos pública de Kaggle de [Partidos del Fútbol Europeo](https://www.kaggle.com/datasets/hugomathien/soccer)

- Checando el nombre de las tablas que tiene la base de datos:

In [2]:
database =  "/app/my_data/canastabasica_datatools/data/raw/database.sqlite"
#"database.sqlite"
#"/Users/renba/Workspace/PYTHON/canastabasica_datatools/data/raw/database.sqlite"
conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table';""", conn) 

conn.close()
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


En este proyecto trabajaremos con las tablas `Player` y `Player_Attributes`:

In [3]:
conn = sqlite3.connect(database)

dict_tables_db = {}
lista_no_tablas = ['sqlite_sequence','Match','League','Country','Team','Team_Attributes']
for tabletmp in tables['name']:
    if tabletmp not in lista_no_tablas:
        query = "SELECT * from %s"%tabletmp
        print(query)
        dict_tables_db[tabletmp] = pd.read_sql_query(query, conn)

conn.close()

[display(x) for x in dict_tables_db.values()]

SELECT * from Player_Attributes
SELECT * from Player


,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,...,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,...,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,...,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


[None, None]

- Creando un pandas `dataframe` para las tablas `Player` y `Player_Attributes`:

In [4]:
df_player_atr = dict_tables_db['Player_Attributes']
df_player_atr.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [5]:
df_player = dict_tables_db['Player']
df_player.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


- Checando los tipos de variables que contienen las tablas:

In [6]:
df_player_atr.dtypes

id                       int64
player_fifa_api_id       int64
player_api_id            int64
date                    object
overall_rating         float64
potential              float64
preferred_foot          object
attacking_work_rate     object
defensive_work_rate     object
crossing               float64
finishing              float64
heading_accuracy       float64
short_passing          float64
volleys                float64
dribbling              float64
curve                  float64
free_kick_accuracy     float64
long_passing           float64
ball_control           float64
acceleration           float64
sprint_speed           float64
agility                float64
reactions              float64
balance                float64
shot_power             float64
jumping                float64
stamina                float64
strength               float64
long_shots             float64
aggression             float64
interceptions          float64
positioning            float64
vision  

In [7]:
df_player.dtypes

id                      int64
player_api_id           int64
player_name            object
player_fifa_api_id      int64
birthday               object
height                float64
weight                  int64
dtype: object

- Realicemos un análisis descriptivo sobre las variables de cada tabla:

In [8]:
for key,tabletbp in dict_tables_db.items():
    print(key)
    print(tabletbp.shape)
    display(tabletbp.describe())

Player_Attributes
(183978, 42)


,id,player_fifa_api_id,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
count,183978.00000,183978.000000,183978.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000,181265.000000,...,181265.000000,183142.000000,183142.000000,183142.000000,181265.000000,183142.000000,183142.000000,183142.000000,183142.000000,183142.000000
mean,91989.50000,165671.524291,135900.617324,68.600015,73.460353,55.086883,49.921078,57.266023,62.429672,49.468436,...,57.873550,55.003986,46.772242,50.351257,48.001462,14.704393,16.063612,20.998362,16.132154,16.441439
std,53110.01825,53851.094769,136927.840510,7.041139,6.592271,17.242135,19.038705,16.488905,14.194068,18.256618,...,15.144086,15.546519,21.227667,21.483706,21.598778,16.865467,15.867382,21.452980,16.099175,17.198155
min,1.00000,2.000000,2625.000000,33.000000,39.000000,1.000000,1.000000,1.000000,3.000000,1.000000,...,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,45995.25000,155798.000000,34763.000000,64.000000,69.000000,45.000000,34.000000,49.000000,57.000000,35.000000,...,49.000000,45.000000,25.000000,29.000000,25.000000,7.000000,8.000000,8.000000,8.000000,8.000000
50%,91989.50000,183488.000000,77741.000000,69.000000,74.000000,59.000000,53.000000,60.000000,65.000000,52.000000,...,60.000000,57.000000,50.000000,56.000000,53.000000,10.000000,11.000000,12.000000,11.000000,11.000000
75%,137983.75000,199848.000000,191080.000000,73.000000,78.000000,68.000000,65.000000,68.000000,72.000000,64.000000,...,69.000000,67.000000,66.000000,69.000000,67.000000,13.000000,15.000000,15.000000,15.000000,15.000000
max,183978.00000,234141.000000,750584.000000,94.000000,97.000000,95.000000,97.000000,98.000000,97.000000,93.000000,...,97.000000,96.000000,96.000000,95.000000,95.000000,94.000000,93.000000,97.000000,96.000000,96.000000


Player
(11060, 7)


,id,player_api_id,player_fifa_api_id,height,weight
count,11060.000000,11060.000000,11060.000000,11060.000000,11060.000000
mean,5537.511392,156582.427215,165664.910488,181.867445,168.380289
std,3197.692647,160713.700624,58649.928360,6.369201,14.990217
min,1.000000,2625.000000,2.000000,157.480000,117.000000
25%,2767.750000,35555.500000,151889.500000,177.800000,159.000000
50%,5536.500000,96619.500000,184671.000000,182.880000,168.000000
75%,8306.250000,212470.500000,203883.250000,185.420000,179.000000
max,11075.000000,750584.000000,234141.000000,208.280000,243.000000


In [9]:
###  Observando los datos de la tabla Player_Attributes, se observa que hay columnas que tienen valores bastante dispersos
###  y que no se entienden a simple vista. Por ejemplo, la columna de porteros (los que empiezan en gk) se observa que cuando tienen ha estar altos los demas attributos tienen ha estar bajos.

### Calculando Promedios de valores de atributos por jugador
### Group by por player_api_id

df_player_atr_means = df_player_atr[['player_api_id',
               'overall_rating',
       'potential', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes',
       ]]\
.groupby('player_api_id').mean()

In [10]:
### Podemos observar que los valores medios de los porteros (los que empiezan con gk) son muchas mas bajos que los de los demas atributos con medias menores a 15.0, mientras que los demas atributos ronda entre 48 y 65.
### Esto nos indica que los porteros tienen atributos muy bajos en comparacion con los demas jugadores del campo. Por lo que para mejorar nuestro modelo de regresión logistica removeremos a los jugadores que parecen ser porteros.


#- Seleccionando columnas de porteros
gk_player=[x for x in df_player_atr.columns if "gk" in x]
gk_player

['gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']


- Hay 836 observaciones con valores faltantes en *overall_rating*
- Para poder validar quitar porteros tambien necesitamos el mean sin NA entonces.

In [11]:
len(df_player_atr[df_player_atr['overall_rating'].isna()])
len(df_player_atr[df_player_atr['gk_diving'].isna()])
len(df_player_atr[df_player_atr['gk_handling'].isna()])
len(df_player_atr[df_player_atr['gk_kicking'].isna()])
len(df_player_atr[df_player_atr['gk_positioning'].isna()])
len(df_player_atr[df_player_atr['gk_reflexes'].isna()])


836

Por lo que se procederán a eliminar dichos valores:

In [12]:
df_player_atr_sin_na =df_player_atr.dropna(subset=['overall_rating'])
df_player_atr_sin_na.head(10)

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,...,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,...,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,...,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
5,6,189615,155782,2016-04-21 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
6,7,189615,155782,2016-04-07 00:00:00,74.0,76.0,left,high,medium,80.0,...,66.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
7,8,189615,155782,2016-01-07 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
8,9,189615,155782,2015-12-24 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0
9,10,189615,155782,2015-12-17 00:00:00,73.0,75.0,left,high,medium,79.0,...,65.0,59.0,76.0,75.0,78.0,14.0,7.0,9.0,9.0,12.0


In [13]:
len(df_player_atr_sin_na[df_player_atr_sin_na['overall_rating'].isna()])

0

In [14]:
df_player_atr = df_player_atr_sin_na

In [15]:

### Diferenciando a los porteros de los jugadores de campo


gk_diving_mean = np.mean(df_player_atr['gk_diving'])
gk_diving_std = np.std(df_player_atr['gk_diving'])
gk_handling_mean = np.mean(df_player_atr['gk_handling'])
gk_handling_std = np.std(df_player_atr['gk_handling'])
gk_kicking_mean = np.mean(df_player_atr['gk_kicking'])
gk_kicking_std = np.std(df_player_atr['gk_kicking'])
gk_positioning_mean = np.mean(df_player_atr['gk_positioning'])
gk_positioning_std = np.std(df_player_atr['gk_positioning'])
gk_reflexes_mean = np.mean(df_player_atr['gk_reflexes'])
gk_reflexes_std = np.std(df_player_atr['gk_reflexes'])


df_player_atr_sin_porteros = df_player_atr_means[(df_player_atr_means['gk_diving']<=(gk_diving_mean+gk_diving_std)) | 
(df_player_atr_means['gk_handling']<=(gk_handling_mean+gk_handling_std)) | 
(df_player_atr_means['gk_kicking']<=(gk_kicking_mean+gk_kicking_std)) | 
(df_player_atr_means['gk_positioning']<=(gk_positioning_mean+gk_positioning_std)) | 
(df_player_atr_means['gk_reflexes']<=(gk_reflexes_mean+gk_reflexes_std))]

df_player_atr_sin_porteros

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
player_api_id,,,,,,,,,,,,,,,,,,,,,
2625,60.142857,61.142857,50.142857,47.285714,46.285714,56.500000,38.000000,54.142857,50.000000,49.928571,...,55.285714,61.928571,62.928571,63.857143,56.571429,12.428571,12.857143,19.071429,10.357143,10.428571
2752,69.380952,70.380952,36.428571,37.428571,75.333333,54.809524,20.428571,44.857143,30.428571,18.857143,...,52.428571,35.428571,71.095238,70.666667,65.571429,11.095238,6.095238,7.095238,9.095238,15.095238
2768,69.285714,70.571429,42.238095,42.761905,66.666667,64.714286,28.761905,40.238095,51.761905,20.047619,...,57.761905,31.904762,71.666667,70.619048,79.714286,10.761905,15.904762,22.714286,15.142857,12.095238
2770,71.133333,73.533333,61.866667,67.666667,67.800000,72.533333,67.133333,75.066667,79.800000,68.666667,...,71.600000,63.933333,30.200000,37.533333,25.000000,8.666667,15.666667,27.200000,16.333333,17.000000
2790,70.200000,75.800000,70.000000,37.400000,55.200000,68.000000,43.000000,60.800000,67.000000,55.400000,...,60.000000,55.400000,74.600000,74.600000,69.000000,8.000000,17.000000,59.000000,16.600000,17.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744907,51.909091,65.909091,42.454545,44.454545,47.454545,59.454545,42.454545,56.454545,46.454545,43.454545,...,51.454545,52.454545,39.454545,48.454545,49.454545,8.454545,13.454545,12.454545,11.454545,8.454545
746419,59.000000,66.000000,55.000000,28.000000,50.000000,35.000000,29.000000,67.000000,37.000000,39.000000,...,40.000000,43.000000,55.000000,69.000000,62.000000,7.000000,10.000000,7.000000,9.000000,9.000000
748432,58.000000,68.000000,48.000000,26.000000,57.000000,49.000000,24.000000,41.000000,39.000000,20.000000,...,36.000000,45.000000,63.000000,69.000000,68.000000,8.000000,8.000000,12.000000,12.000000,6.000000


In [16]:
df_player_atr_sin_porteros.columns

Index(['overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes'],
      dtype='object')

In [17]:
### Quitar valores de porteros
df_player_atr_sin_porteros = df_player_atr_sin_porteros.drop(columns = ['gk_diving','gk_handling','gk_kicking','gk_positioning','gk_reflexes'])

df_player_atr_means = df_player_atr_sin_porteros

In [18]:
df_player_atr_sin_porteros.columns

Index(['overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle'],
      dtype='object')

De acuerdo a la FIFA, considera a los jugadores con un overall_rating mayor a 75 como posibles candidatos para el salon de la fama para someterse a una votación.

Como target a predecir se utilizará la siguiente:
- Entra_salon_fama: toma el valor 1 si el promedio del overall_rating del jugador es mayor a 75 y 0 en caso contrario. Que para la fifa eso es un valor coeficiente de un buen jugador.

In [19]:

df_final = df_player[['id','player_api_id','player_name','player_fifa_api_id','height','weight']].merge(
    df_player_atr_means,
    how='right',left_on='player_api_id', right_on = 'player_api_id'
).assign(
   entra_salon_fama = lambda x: np.where(x['overall_rating']>=75,1,0),
)

In [20]:
df_final[['entra_salon_fama']].value_counts()

entra_salon_fama
0                   9133
1                    968
Name: count, dtype: int64

In [21]:
df_final.to_csv('data_for_traintest.csv', index=False)
### Maria

### Métricas de evaluación

#### Clasificación binaria

La Exactitud (Accuracy) es simplemente la proporción de datos clasificados correctamente. 

\begin{align*}
    Accuracy = \frac{VP+VN}{TOTAL} 
\end{align*}

donde $VP$ es el número de predicciones positivas correctamente clasificadas, $VN$ es el número de predicciones negativas correctamente clasificadas y $TOTAL$ es el número total de casos u observaciones.

Es la primera métrica que se comprueba al evaluar un clasificador; sin embargo, si los datos están desbalanceados o si se está más interesado detectar una de las clases, la Exactitud no captura realmente la eficacia de un clasificador.

La precisión (Precision) nos ayuda a cuantificar la fracción de positivos verdaderos entre el total de los clasificados como positivos. Se utiliza principalmente para medir qué tan efectivo es el modelo en detectar la categoría de interés, es decir, la categoría positiva.

\begin{align*}
  Precision = \frac{VP}{VP+FP} 
\end{align*}

donde $FP$ es el número de predicciones positivas calificadas incorrectamente (falsos positivos). 

La recuperación (Recall) mide cuántos positivos verdaderos se predijeron como positivos. 

\begin{align*}
    Recall = \frac{VP}{VP+FN} 
\end{align*}

donde $FN$ es el número de predicciones negativas calificadas incorrectamente.

Observa que mientras más cercano a 1 se encuentre el recall, más datos de la categoría real fueron bien clasificados. 

La medida F1 (F1 score) es la media armónica de la precisión y la recuperación (Recall). Esta medida es una buena forma de resumir la evaluación en un número único.

\begin{align*}
    F1 = 2\cdot \frac{precision \cdot recall}{precision + recall}
\end{align*}

Existe una relación entre la precisión y el recall: un modelo que predice todo como positivo tendrá un recall de 1, pero una precisión muy baja pues tendría muchos falsos positivos, mientras que un modelo que solo predijera un positivo y el resto negativos tendría un recall muy bajo, pero una precisión muy alta. 

Es por ello que se recurre a la medida F1, ya que mitiga el impacto de las tasas altas y acentúa el de las tasas bajas.

In [22]:
data_with_target = pd.read_csv('data_for_traintest.csv')
data_with_target

,id,player_api_id,player_name,player_fifa_api_id,height,weight,overall_rating,potential,crossing,finishing,...,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,entra_salon_fama
0,8372,2625,"Patryk Rachwal,18",148544,175.26,154,60.142857,61.142857,50.142857,47.285714,...,56.857143,71.285714,62.071429,50.357143,55.285714,61.928571,62.928571,63.857143,56.571429,0
1,2639,2752,Diego Mainz,135819,187.96,174,69.380952,70.380952,36.428571,37.428571,...,19.571429,80.285714,71.428571,31.428571,52.428571,35.428571,71.095238,70.666667,65.571429,0
2,5319,2768,Jose Dorado,110019,180.34,154,69.285714,70.571429,42.238095,42.761905,...,19.428571,62.761905,74.714286,44.190476,57.761905,31.904762,71.666667,70.619048,79.714286,0
3,4339,2770,Ignacio Gonzalez,182861,180.34,161,71.133333,73.533333,61.866667,67.666667,...,73.000000,56.800000,44.933333,64.533333,71.600000,63.933333,30.200000,37.533333,25.000000,0
4,306,2790,Alberto Rey,110809,175.26,163,70.200000,75.800000,70.000000,37.400000,...,42.400000,68.400000,63.400000,60.600000,60.000000,55.400000,74.600000,74.600000,69.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10096,3982,744907,Greg Taylor,226508,172.72,137,51.909091,65.909091,42.454545,44.454545,...,38.454545,36.454545,22.454545,48.454545,51.454545,52.454545,39.454545,48.454545,49.454545,0
10097,3488,746419,Francisco Afonso,233930,175.26,152,59.000000,66.000000,55.000000,28.000000,...,27.000000,59.000000,53.000000,55.000000,40.000000,43.000000,55.000000,69.000000,62.000000,0
10098,3607,748432,Fynn Arkenberg,233969,187.96,176,58.000000,68.000000,48.000000,26.000000,...,25.000000,52.000000,55.000000,38.000000,36.000000,45.000000,63.000000,69.000000,68.000000,0
10099,8841,750435,Rees Greenwood,225462,172.72,150,56.444444,70.444444,35.000000,57.000000,...,50.000000,34.000000,13.000000,56.333333,58.000000,61.000000,18.000000,19.000000,21.000000,0


In [23]:
data_with_target.columns

Index(['id', 'player_api_id', 'player_name', 'player_fifa_api_id', 'height',
       'weight', 'overall_rating', 'potential', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve',
       'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration',
       'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power',
       'jumping', 'stamina', 'strength', 'long_shots', 'aggression',
       'interceptions', 'positioning', 'vision', 'penalties', 'marking',
       'standing_tackle', 'sliding_tackle', 'entra_salon_fama'],
      dtype='object')

In [24]:
data_with_target=data_with_target.drop(columns=['overall_rating','potential', 'id', 'player_api_id', 'player_name', 'player_fifa_api_id'])
data_with_target.columns

Index(['height', 'weight', 'crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy',
       'long_passing', 'ball_control', 'acceleration', 'sprint_speed',
       'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina',
       'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'entra_salon_fama'],
      dtype='object')

In [25]:
import math
n=math.floor(.80*(len(data_with_target)))
n

8080

In [26]:
data_with_target_X_train = data_with_target.sample(n=n, random_state=123).fillna(method='ffill').fillna(-99)
data_with_target_X_train

,height,weight,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,entra_salon_fama
221,177.80,161,54.111111,71.555556,65.333333,64.777778,74.000000,73.444444,63.333333,70.777778,...,69.000000,67.000000,41.666667,71.555556,52.777778,68.777778,37.444444,35.000000,30.444444,0
9591,185.42,165,44.714286,67.142857,67.000000,54.285714,55.000000,71.000000,39.000000,38.000000,...,62.000000,63.142857,13.000000,65.000000,52.428571,59.000000,19.000000,13.000000,14.000000,0
8953,172.72,157,64.500000,44.000000,45.500000,57.900000,43.500000,58.600000,58.000000,58.500000,...,40.500000,69.000000,60.700000,56.500000,50.500000,34.500000,54.100000,57.200000,61.000000,0
5943,182.88,163,58.950000,53.900000,59.550000,74.900000,54.200000,60.950000,69.250000,65.550000,...,56.150000,43.000000,52.150000,64.150000,68.850000,63.450000,38.350000,44.000000,32.400000,0
4962,180.34,190,52.000000,48.000000,55.000000,59.000000,54.200000,52.000000,69.250000,53.000000,...,46.000000,80.000000,49.000000,55.000000,68.850000,54.000000,51.750000,52.500000,32.400000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,190.50,174,52.500000,53.333333,79.541667,75.250000,57.083333,61.750000,34.083333,42.583333,...,57.916667,82.333333,79.541667,63.125000,71.916667,64.166667,73.458333,80.416667,75.833333,1
2429,185.42,165,74.736842,63.368421,61.157895,79.684211,74.315789,71.947368,71.157895,67.736842,...,52.947368,60.473684,54.105263,69.000000,77.473684,71.368421,23.315789,24.736842,28.947368,0
9038,198.12,168,40.000000,61.578947,65.000000,49.000000,55.000000,64.368421,52.000000,54.000000,...,55.684211,53.157895,41.000000,56.157895,44.000000,56.052632,19.210526,22.000000,20.789474,0
5563,193.04,192,44.952381,65.523810,62.285714,54.619048,53.000000,65.428571,52.000000,42.952381,...,58.380952,61.142857,30.238095,59.761905,37.333333,46.380952,31.952381,32.952381,29.000000,0


In [27]:
data_with_target_Y_train = data_with_target.sample(n=n, random_state=123)["entra_salon_fama"]
data_with_target_Y_train

221     0
9591    0
8953    0
5943    0
4962    0
       ..
3384    1
2429    0
9038    0
5563    0
9900    0
Name: entra_salon_fama, Length: 8080, dtype: int64

In [28]:
data_with_target_X_test= data_with_target.drop(data_with_target_X_train.index).fillna(method='ffill').fillna(-99)
data_with_target_X_test

,height,weight,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,entra_salon_fama
13,175.26,150,57.000000,53.857143,57.571429,69.000000,54.000000,60.428571,63.000000,51.571429,...,57.000000,64.857143,68.000000,50.428571,68.000000,60.142857,57.000000,62.000000,61.000000,0
16,185.42,165,49.000000,64.000000,63.400000,54.000000,58.000000,56.400000,46.000000,44.000000,...,59.000000,52.000000,56.400000,66.000000,66.000000,57.800000,30.000000,34.000000,36.000000,0
17,180.34,159,72.692308,53.384615,63.615385,62.846154,42.230769,61.384615,57.615385,62.461538,...,55.153846,72.153846,57.384615,56.769231,55.384615,55.923077,63.076923,68.384615,67.000000,0
22,175.26,168,72.238095,78.619048,70.333333,76.000000,68.952381,72.857143,66.142857,76.190476,...,77.809524,78.714286,44.666667,82.476190,75.380952,80.380952,38.142857,21.095238,20.571429,1
26,177.80,154,57.545455,55.409091,55.136364,76.227273,57.954545,73.590909,70.272727,60.818182,...,61.409091,78.954545,73.636364,62.954545,71.909091,56.409091,53.681818,69.772727,63.636364,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10069,193.04,198,32.000000,23.125000,69.000000,50.375000,28.000000,32.500000,28.000000,34.000000,...,21.000000,60.750000,57.750000,29.375000,37.000000,44.000000,63.250000,67.625000,70.000000,0
10087,193.04,165,23.000000,58.000000,70.000000,60.000000,63.000000,48.000000,30.000000,32.000000,...,50.000000,17.000000,22.000000,55.000000,50.000000,55.000000,23.000000,18.000000,23.000000,0
10089,177.80,161,36.000000,31.000000,46.000000,27.000000,30.000000,53.000000,33.000000,30.000000,...,28.000000,46.000000,51.000000,40.000000,35.000000,38.000000,48.000000,59.000000,57.000000,0
10094,175.26,172,40.000000,32.000000,50.000000,55.000000,40.000000,48.000000,38.000000,38.000000,...,39.000000,58.000000,46.000000,47.000000,47.000000,42.000000,38.000000,60.000000,55.000000,0


In [29]:
data_with_target_Y_test = data_with_target.drop(data_with_target_X_train.index)["entra_salon_fama"]
data_with_target_Y_test

13       0
16       0
17       0
22       1
26       0
        ..
10069    0
10087    0
10089    0
10094    0
10099    0
Name: entra_salon_fama, Length: 2021, dtype: int64

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
#from category_encoders import OneHotEncoder
X_train, X_test, y_train, y_test = train_test_split(data_with_target_X_train, data_with_target_Y_train, test_size=0.2, random_state=42)


In [31]:
# Instantiate the logistic regression model
log_reg = LogisticRegression()

# Fit the model to the training data
log_reg.fit(X_train, y_train)


/opt/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [32]:
# Predict the outcomes for the testing data
y_pred = log_reg.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9733910891089109


In [33]:
from sklearn import metrics
y_pred_proba = log_reg.predict_proba(X_test)[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = round(metrics.roc_auc_score(y_test, y_pred_proba),4)
auc

0.9864

In [34]:
confusion_matrix = metrics.confusion_matrix(y_test, y_pred)
print(confusion_matrix)

[[1455   13]
 [  30  118]]


<center> Predicciones

|  | No | Si |
|--|--|--|
|No|1456|13|
|Si|30|118|

In [35]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      1468
           1       0.90      0.80      0.85       148

    accuracy                           0.97      1616
   macro avg       0.94      0.89      0.92      1616
weighted avg       0.97      0.97      0.97      1616



In [36]:
#Predict the outcomes for the testing data
y_pred_test = log_reg.predict(data_with_target_X_test)

#Evaluate the model's performance
accuracy_test=accuracy_score(data_with_target_Y_test, y_pred_test)
print("Accuracy:", accuracy_test)

Accuracy: 0.9807026224641266


In [37]:
confusion_matrix = metrics.confusion_matrix(data_with_target_Y_test, y_pred_test)
print(confusion_matrix)

[[1811   18]
 [  21  171]]


<center> Predicciones

|  | No | Si |
|--|--|--|
|No|1811|18|
|Si|21|171|

In [38]:
from sklearn.metrics import f1_score
f1_score2 = f1_score(data_with_target_Y_test, y_pred_test)
print(f1_score2)

0.8976377952755906


In [39]:
print(metrics.classification_report(data_with_target_Y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1829
           1       0.90      0.89      0.90       192

    accuracy                           0.98      2021
   macro avg       0.95      0.94      0.94      2021
weighted avg       0.98      0.98      0.98      2021



### ¿Escala?

Todos los valores están en la escala de 1 al 100, excepto 'height'y 'weight'

In [40]:
data_with_target.describe()

,height,weight,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,...,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,entra_salon_fama
count,10101.000000,10101.000000,10101.000000,10101.000000,10101.000000,10101.000000,9665.000000,10101.000000,9665.000000,10101.000000,...,10101.000000,10101.000000,10101.000000,10101.000000,9665.000000,10101.000000,10101.000000,10101.000000,9665.000000,10101.000000
mean,181.218854,166.858232,56.066507,50.668263,59.574464,63.447197,49.943360,60.480421,53.533328,50.079381,...,53.863299,61.698023,53.360730,57.024138,58.183811,55.934754,48.846804,52.298879,49.891074,0.095832
std,6.118472,14.276079,12.864767,16.329444,11.272991,9.275658,15.316226,12.694439,14.634672,14.242820,...,14.741319,13.110992,16.393548,13.744601,12.127357,11.894800,18.963730,18.878561,19.321790,0.294375
min,157.480000,117.000000,12.000000,10.000000,14.333333,19.461538,10.000000,14.000000,13.000000,7.000000,...,10.000000,11.000000,10.000000,9.333333,13.000000,10.000000,10.000000,10.000000,10.000000,0.000000
25%,177.800000,157.000000,47.571429,37.051282,52.200000,58.000000,38.000000,53.520000,43.000000,39.250000,...,44.153846,54.000000,41.083333,49.000000,50.625000,47.363636,29.750000,34.153846,30.888889,0.000000
50%,180.340000,165.000000,57.900000,52.944444,60.111111,64.000000,51.586207,62.526316,54.200000,50.000000,...,56.583333,63.714286,57.250000,59.250000,59.000000,56.400000,54.000000,58.566667,56.000000,0.000000
75%,185.420000,176.000000,65.400000,64.000000,67.500000,69.666667,61.708333,69.450000,64.560000,60.846154,...,64.714286,71.285714,66.000000,66.833333,66.900000,64.681818,65.000000,67.789474,65.928571,0.000000
max,203.200000,225.000000,89.357143,92.230769,93.111111,95.181818,90.789474,96.461538,92.571429,90.488889,...,90.000000,93.666667,91.750000,93.200000,95.681818,92.000000,90.666667,90.200000,94.366667,1.000000


In [41]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)

In [42]:
# Instantiate the logistic regression model
log_reg_scaled = LogisticRegression()

# Fit the model to the training data
log_reg_scaled.fit(X_train_scaled, y_train)

LogisticRegression()

In [43]:
X_test_scaled = scaler.fit_transform(X_test)
# Predict the outcomes for the testing data
y_pred_scaled = log_reg_scaled.predict(X_test_scaled)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred_scaled)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [44]:
y_pred_scaled_proba = log_reg.predict_proba(X_test_scaled)[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_scaled_proba)
auc = round(metrics.roc_auc_score(y_test, y_pred_scaled_proba),4)
auc

/opt/venv/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


1.0

In [45]:
X_test_scaled = scaler.fit_transform(data_with_target_X_test)
# Predict the outcomes for the testing data
y_pred_test_scaled = log_reg_scaled.predict(X_test_scaled)

# Evaluate the model's performance
accuracy_test_scaled = accuracy_score(data_with_target_Y_test, y_pred_test_scaled)
print("Accuracy:", accuracy_test_scaled)

Accuracy: 1.0


In [46]:
confusion_matrix = metrics.confusion_matrix(data_with_target_Y_test, y_pred_test_scaled)
print(confusion_matrix)

[[1829    0]
 [   0  192]]


<center> Predicciones 

|  | No | Si |
|--|--|--|
|No|1829|0|
|Si|0|192|

In [47]:
print(metrics.classification_report(data_with_target_Y_test, y_pred_test_scaled))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1829
           1       1.00      1.00      1.00       192

    accuracy                           1.00      2021
   macro avg       1.00      1.00      1.00      2021
weighted avg       1.00      1.00      1.00      2021



### ¿Y si quitamos 'height', 'weight'?

In [48]:
data_with_target2=data_with_target.drop(columns=['height', 'weight'])
data_with_target2.columns

Index(['crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys',
       'dribbling', 'curve', 'free_kick_accuracy', 'long_passing',
       'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions',
       'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'aggression', 'interceptions', 'positioning', 'vision', 'penalties',
       'marking', 'standing_tackle', 'sliding_tackle', 'entra_salon_fama'],
      dtype='object')

In [49]:
data_with_target_X_train2 = data_with_target2.sample(n=n, random_state=123).fillna(method='ffill').fillna(-99)
data_with_target_X_train2

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,...,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,entra_salon_fama
221,54.111111,71.555556,65.333333,64.777778,74.000000,73.444444,63.333333,70.777778,50.555556,74.444444,...,69.000000,67.000000,41.666667,71.555556,52.777778,68.777778,37.444444,35.000000,30.444444,0
9591,44.714286,67.142857,67.000000,54.285714,55.000000,71.000000,39.000000,38.000000,38.428571,67.857143,...,62.000000,63.142857,13.000000,65.000000,52.428571,59.000000,19.000000,13.000000,14.000000,0
8953,64.500000,44.000000,45.500000,57.900000,43.500000,58.600000,58.000000,58.500000,50.500000,60.300000,...,40.500000,69.000000,60.700000,56.500000,50.500000,34.500000,54.100000,57.200000,61.000000,0
5943,58.950000,53.900000,59.550000,74.900000,54.200000,60.950000,69.250000,65.550000,70.500000,64.800000,...,56.150000,43.000000,52.150000,64.150000,68.850000,63.450000,38.350000,44.000000,32.400000,0
4962,52.000000,48.000000,55.000000,59.000000,54.200000,52.000000,69.250000,53.000000,47.000000,58.000000,...,46.000000,80.000000,49.000000,55.000000,68.850000,54.000000,51.750000,52.500000,32.400000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3384,52.500000,53.333333,79.541667,75.250000,57.083333,61.750000,34.083333,42.583333,72.250000,72.750000,...,57.916667,82.333333,79.541667,63.125000,71.916667,64.166667,73.458333,80.416667,75.833333,1
2429,74.736842,63.368421,61.157895,79.684211,74.315789,71.947368,71.157895,67.736842,71.263158,75.105263,...,52.947368,60.473684,54.105263,69.000000,77.473684,71.368421,23.315789,24.736842,28.947368,0
9038,40.000000,61.578947,65.000000,49.000000,55.000000,64.368421,52.000000,54.000000,37.000000,65.315789,...,55.684211,53.157895,41.000000,56.157895,44.000000,56.052632,19.210526,22.000000,20.789474,0
5563,44.952381,65.523810,62.285714,54.619048,53.000000,65.428571,52.000000,42.952381,53.142857,66.476190,...,58.380952,61.142857,30.238095,59.761905,37.333333,46.380952,31.952381,32.952381,29.000000,0


In [50]:
data_with_target_Y_train2 = data_with_target2.sample(n=n, random_state=123)["entra_salon_fama"]
data_with_target_Y_train2

221     0
9591    0
8953    0
5943    0
4962    0
       ..
3384    1
2429    0
9038    0
5563    0
9900    0
Name: entra_salon_fama, Length: 8080, dtype: int64

In [51]:
data_with_target_Y_test2 = data_with_target2.drop(data_with_target_X_train2.index)["entra_salon_fama"]
data_with_target_Y_test2

13       0
16       0
17       0
22       1
26       0
        ..
10069    0
10087    0
10089    0
10094    0
10099    0
Name: entra_salon_fama, Length: 2021, dtype: int64

In [52]:
data_with_target_X_test2= data_with_target2.drop(data_with_target_X_train2.index).fillna(method='ffill').fillna(-99)
data_with_target_X_test2

,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,...,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,entra_salon_fama
13,57.000000,53.857143,57.571429,69.000000,54.000000,60.428571,63.000000,51.571429,67.142857,66.428571,...,57.000000,64.857143,68.000000,50.428571,68.000000,60.142857,57.000000,62.000000,61.000000,0
16,49.000000,64.000000,63.400000,54.000000,58.000000,56.400000,46.000000,44.000000,27.000000,62.800000,...,59.000000,52.000000,56.400000,66.000000,66.000000,57.800000,30.000000,34.000000,36.000000,0
17,72.692308,53.384615,63.615385,62.846154,42.230769,61.384615,57.615385,62.461538,67.384615,62.000000,...,55.153846,72.153846,57.384615,56.769231,55.384615,55.923077,63.076923,68.384615,67.000000,0
22,72.238095,78.619048,70.333333,76.000000,68.952381,72.857143,66.142857,76.190476,68.142857,79.238095,...,77.809524,78.714286,44.666667,82.476190,75.380952,80.380952,38.142857,21.095238,20.571429,1
26,57.545455,55.409091,55.136364,76.227273,57.954545,73.590909,70.272727,60.818182,69.000000,78.954545,...,61.409091,78.954545,73.636364,62.954545,71.909091,56.409091,53.681818,69.772727,63.636364,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10069,32.000000,23.125000,69.000000,50.375000,28.000000,32.500000,28.000000,34.000000,45.875000,51.750000,...,21.000000,60.750000,57.750000,29.375000,37.000000,44.000000,63.250000,67.625000,70.000000,0
10087,23.000000,58.000000,70.000000,60.000000,63.000000,48.000000,30.000000,32.000000,30.000000,60.000000,...,50.000000,17.000000,22.000000,55.000000,50.000000,55.000000,23.000000,18.000000,23.000000,0
10089,36.000000,31.000000,46.000000,27.000000,30.000000,53.000000,33.000000,30.000000,30.000000,30.000000,...,28.000000,46.000000,51.000000,40.000000,35.000000,38.000000,48.000000,59.000000,57.000000,0
10094,40.000000,32.000000,50.000000,55.000000,40.000000,48.000000,38.000000,38.000000,55.000000,48.000000,...,39.000000,58.000000,46.000000,47.000000,47.000000,42.000000,38.000000,60.000000,55.000000,0


In [53]:
data_with_target_Y_test2 = data_with_target2.drop(data_with_target_X_train2.index)["entra_salon_fama"]
data_with_target_Y_test2

13       0
16       0
17       0
22       1
26       0
        ..
10069    0
10087    0
10089    0
10094    0
10099    0
Name: entra_salon_fama, Length: 2021, dtype: int64

In [54]:

#from category_encoders import OneHotEncoder
X_train2, X_test2, y_train2, y_test2 = train_test_split(data_with_target_X_train2, data_with_target_Y_train2, test_size=0.2, random_state=42)


In [55]:
# Instantiate the logistic regression model
log_reg2 = LogisticRegression()

# Fit the model to the training data
log_reg2.fit(X_train2, y_train2)


/opt/venv/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [56]:
# Predict the outcomes for the testing data
y_pred2 = log_reg2.predict(X_test2)

# Evaluate the model's performance
accuracy2 = accuracy_score(y_test2, y_pred2)
print("Accuracy:", accuracy2)

Accuracy: 0.9900990099009901


In [57]:
from sklearn import metrics
y_pred_proba2 = log_reg2.predict_proba(X_test2)[:,1]
fpr, tpr, _ = metrics.roc_curve(y_test2,  y_pred_proba2)
auc2 = round(metrics.roc_auc_score(y_test2, y_pred_proba2),4)
auc2

0.9983

In [58]:
from sklearn.metrics import f1_score

In [59]:
f1_score2 = f1_score(y_test2,y_pred2)
print(f1_score2)

0.9473684210526315


In [60]:
confusion_matrix2 = metrics.confusion_matrix(y_test2, y_pred2)
print(confusion_matrix2)

[[1456   12]
 [   4  144]]


<center> Predicciones 

|  | No| Si |
|--|--|--|
|No|1456|12|
|Si|4|144|

In [61]:
print(metrics.classification_report(y_test2, y_pred2))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1468
           1       0.92      0.97      0.95       148

    accuracy                           0.99      1616
   macro avg       0.96      0.98      0.97      1616
weighted avg       0.99      0.99      0.99      1616



## Conclusiones

En este proyecto se trabajó sobre prediciones utilizando un modelo de regresión logística en el que si el promedio del atributos *overall rating* durante la carrera del jugador es mayor a 75 entonces este es candidato a las votaciones para entrar al salón de la fama. 

- Se realizó el análisis de la base de datos, en la que se procedió a eliminar los valores ausentes de la variable *overall rating*.
-  Observamos variables relacionadas con atributos de porteros, por lo que se estableció un criterio de decisión para eliminar a dichos jugadores y se trabajó con 10,101 observaciones.
-  En el modelo observamos 


### Trabajos a futuro

- Realizar un análisis de los porteros.
- Comparar el desempeño de los jugadores con los partidos (Match) considerando los atributos de los mismos.